In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

DATA_COLUMNS = ["A","B","C","D","E","F","G","DIGIT"]
leds = np.array([[1,1,1,1,1,1,0],
                     [0,1,1,0,0,0,0],
                     [1,1,0,1,1,0,1],
                     [1,1,1,1,0,0,1],
                     [0,0,1,0,0,1,1],
                     [1,0,1,1,0,1,1],
                     [1,0,1,1,1,1,1],
                     [1,1,1,0,0,0,0],
                     [1,1,1,1,1,1,1],
                     [1,1,1,1,0,1,1]
                    ])

input_data = []
for indx in range(len(leds)):
    led = [np.append(leds[indx], indx)]
    if len(input_data) == 0:
        input_data = led
    else:
        input_data = np.concatenate((input_data, led), axis=0)

In [2]:
leds_strength = np.sum(leds, axis=1)

In [3]:
leds_importance = np.divide(leds, leds_strength[:,None])
total_leds_importance = np.sum(leds_importance, axis=0)

In [4]:
leds_relative_importance = np.divide(leds_importance, total_leds_importance)

In [5]:
for indx in range(len(leds_relative_importance)):
    led = leds_relative_importance[indx]
    print(indx, led)

0 [0.10574018 0.09749304 0.07543103 0.13409962 0.24647887 0.1417004
 0.        ]
1 [0.         0.29247911 0.2262931  0.         0.         0.
 0.        ]
2 [0.12688822 0.11699164 0.         0.16091954 0.29577465 0.
 0.14189189]
3 [0.12688822 0.11699164 0.09051724 0.16091954 0.         0.
 0.14189189]
4 [0.         0.         0.15086207 0.         0.         0.28340081
 0.23648649]
5 [0.12688822 0.         0.09051724 0.16091954 0.         0.17004049
 0.14189189]
6 [0.10574018 0.         0.07543103 0.13409962 0.24647887 0.1417004
 0.11824324]
7 [0.21148036 0.19498607 0.15086207 0.         0.         0.
 0.        ]
8 [0.09063444 0.08356546 0.06465517 0.11494253 0.21126761 0.12145749
 0.10135135]
9 [0.10574018 0.09749304 0.07543103 0.13409962 0.         0.1417004
 0.11824324]


In [6]:
def find_2_least_important_segments(l):
    leds = l.copy()
    leds[leds == 0] = 2
    least_important_segments = []
    for indx in range(len(leds)):
        led = leds[indx]
        seg1 = led.argmin()
        led[seg1] = 2
        seg2 = led.argmin()
        least_important_segments.append((seg1, seg2))
        
    return least_important_segments 
    

In [7]:
least_imp_segs = find_2_least_important_segments(leds_relative_importance)

In [8]:
least_imp_segs

[(2, 1),
 (2, 1),
 (1, 0),
 (2, 1),
 (2, 6),
 (2, 0),
 (2, 0),
 (2, 1),
 (2, 1),
 (2, 1)]

In [48]:
def stringify_array(arr):
    arr = arr.flatten()
    array_str = None
    for val in arr:
        if array_str == None:
            array_str = str(val)
        else:
            array_str = array_str + "," + str(val)
    
    return array_str

In [49]:
dict = {}
for indx in range(len(leds)):
    (seg1, seg2) = least_imp_segs[indx]
    led = leds[indx]
    v = led[seg1]
    led[seg1] = 0
    led_str = stringify_array(led)
    if dict.__contains__(led_str):
        print("Clash1", led_str, dict.get(led_str), indx, seg1)
        if leds_relative_importance[dict.get(led_str)][seg1] < leds_relative_importance[indx][seg1]:
            print("Updating dict")
            dict[led_str] = indx
    else:
        dict[led_str] = indx
    led[seg1] = v
    
    v = led[seg2]
    led[seg2] = 0
    led_str = stringify_array(led)
    if dict.__contains__(led_str):
        print("Clash2", led_str, dict.get(led_str), indx, seg2)
        if leds_relative_importance[dict.get(led_str)][seg2] < leds_relative_importance[indx][seg2]:
            print("Updating dict")
            dict[led_str] = indx
    else:
        dict[led_str] = indx
    led[seg2] = v


Clash2 1,0,0,1,1,1,1 6 8 1
Updating dict
Clash2 1,0,0,1,0,1,1 5 9 1
Updating dict


In [50]:
dict

{'0,0,0,0,0,0,0': 1,
 '0,0,0,0,0,1,0': 4,
 '0,0,0,0,0,1,1': 4,
 '0,0,0,1,0,1,1': 5,
 '0,0,0,1,1,0,1': 2,
 '0,0,0,1,1,1,1': 6,
 '0,1,0,0,0,0,0': 1,
 '1,0,0,0,0,0,0': 7,
 '1,0,0,1,0,0,1': 3,
 '1,0,0,1,0,1,1': 9,
 '1,0,0,1,1,0,1': 2,
 '1,0,0,1,1,1,0': 0,
 '1,0,0,1,1,1,1': 8,
 '1,1,0,0,0,0,0': 7,
 '1,1,0,1,0,0,1': 3,
 '1,1,0,1,0,1,1': 9,
 '1,1,0,1,1,1,0': 0,
 '1,1,0,1,1,1,1': 8}

In [54]:
for led_str in dict.keys():
    led = np.fromstring(led_str, dtype=int, sep=',')
    led = [np.append(led, dict.get(led_str))]
    input_data = np.concatenate((input_data, led), axis=0)

[array([1, 1, 0, 1, 1, 1, 0, 0])]
[array([1, 0, 0, 1, 1, 1, 0, 0])]
[array([0, 1, 0, 0, 0, 0, 0, 1])]
[array([0, 0, 0, 0, 0, 0, 0, 1])]
[array([1, 0, 0, 1, 1, 0, 1, 2])]
[array([0, 0, 0, 1, 1, 0, 1, 2])]
[array([1, 1, 0, 1, 0, 0, 1, 3])]
[array([1, 0, 0, 1, 0, 0, 1, 3])]
[array([0, 0, 0, 0, 0, 1, 1, 4])]
[array([0, 0, 0, 0, 0, 1, 0, 4])]
[array([1, 0, 0, 1, 0, 1, 1, 9])]
[array([0, 0, 0, 1, 0, 1, 1, 5])]
[array([1, 0, 0, 1, 1, 1, 1, 8])]
[array([0, 0, 0, 1, 1, 1, 1, 6])]
[array([1, 1, 0, 0, 0, 0, 0, 7])]
[array([1, 0, 0, 0, 0, 0, 0, 7])]
[array([1, 1, 0, 1, 1, 1, 1, 8])]
[array([1, 1, 0, 1, 0, 1, 1, 9])]


In [55]:
input_data

array([[1, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 2],
       [1, 1, 1, 1, 0, 0, 1, 3],
       [0, 0, 1, 0, 0, 1, 1, 4],
       [1, 0, 1, 1, 0, 1, 1, 5],
       [1, 0, 1, 1, 1, 1, 1, 6],
       [1, 1, 1, 0, 0, 0, 0, 7],
       [1, 1, 1, 1, 1, 1, 1, 8],
       [1, 1, 1, 1, 0, 1, 1, 9],
       [1, 1, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 2],
       [0, 0, 0, 1, 1, 0, 1, 2],
       [1, 1, 0, 1, 0, 0, 1, 3],
       [1, 0, 0, 1, 0, 0, 1, 3],
       [0, 0, 0, 0, 0, 1, 1, 4],
       [0, 0, 0, 0, 0, 1, 0, 4],
       [1, 0, 0, 1, 0, 1, 1, 9],
       [0, 0, 0, 1, 0, 1, 1, 5],
       [1, 0, 0, 1, 1, 1, 1, 8],
       [0, 0, 0, 1, 1, 1, 1, 6],
       [1, 1, 0, 0, 0, 0, 0, 7],
       [1, 0, 0, 0, 0, 0, 0, 7],
       [1, 1, 0, 1, 1, 1, 1, 8],
       [1, 1, 0, 1, 0, 1, 1, 9]])

In [46]:
stringify_array(input_data[0])

'1,1,1,1,1,1,0,0'

In [47]:
np.fromstring(stringify_array(input_data[0]), dtype=int, sep=',')

array([1, 1, 1, 1, 1, 1, 0, 0])